<a href="https://colab.research.google.com/github/OlgaAcosta/Project1_DataScience_Henry/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Extraigo la data de los archivos

In [ ]:
# Guardo los archivos con los datos en dataframes:
import pandas as pd

movies=pd.read_csv("/content/drive/MyDrive/PI_ML_OPS/Dataset/movies_dataset.csv", sep=",")
credits=pd.read_csv("/content/drive/MyDrive/PI_ML_OPS/Dataset/credits.csv", sep=",")
credits.head()

<ipython-input-71-7390b6f1a5ef>:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies=pd.read_csv("/content/drive/MyDrive/PI_ML_OPS/Dataset/movies_dataset.csv", sep=",")


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


# I. Comienzo con los datos de "credit"


In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
credits["crew"][0]

'[{\'credit_id\': \'52fe4284c3a36847f8024f49\', \'department\': \'Directing\', \'gender\': 2, \'id\': 7879, \'job\': \'Director\', \'name\': \'John Lasseter\', \'profile_path\': \'/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f4f\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12891, \'job\': \'Screenplay\', \'name\': \'Joss Whedon\', \'profile_path\': \'/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f55\', \'department\': \'Writing\', \'gender\': 2, \'id\': 7, \'job\': \'Screenplay\', \'name\': \'Andrew Stanton\', \'profile_path\': \'/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f5b\', \'department\': \'Writing\', \'gender\': 2, \'id\': 12892, \'job\': \'Screenplay\', \'name\': \'Joel Cohen\', \'profile_path\': \'/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg\'}, {\'credit_id\': \'52fe4284c3a36847f8024f61\', \'department\': \'Writing\', \'gender\': 0, \'id\': 12893, \'job\': \'Screenplay\', \'name\': \'A

El tipo de dato de cada elemento de la columna "crew" es un string de una lista de diccionarios.
Uso el método "ast.literal_eval" para convertir la columna "crew" en una lista:

In [ ]:
# Importo la librería a utilizar
import ast

#Creo una función para aplicar "ast.literal_eval", en donde controlo los "nulos" (ya que, si los hay, me bota error):
def app_ast(columna_valor):
  if pd.notnull(columna_valor):
    try:
      return ast.literal_eval(columna_valor)
    except (ValueError, SyntaxError):
      return None
  else:
      return None



In [ ]:
# Aplico la función a  "credits"
credits["crew"]=credits["crew"].apply(app_ast)
#credits["crew"] = credits['crew'].apply(ast.literal_eval)

In [ ]:
# Ahora cada elemento ya es una lista de diccionarios:
type(credits["crew"][0])

list

Creo una función que recorra cada diccionario de la lista "crew" y me devuelva una lista con los directores de cada película:

In [ ]:
def get_directors(crew):
    directors=[]
    for member in crew:
        if member["job"]=="Director":
            directors.append(member["name"])
    return directors

# Creo una nueva columna en el df "credits" aplicando la función "get_directors" a cada registro de la columna "crew":
credits["director"]= credits["crew"].apply(get_directors)

# Muestro:

credits.head()

,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,[John Lasseter]
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,[Joe Johnston]
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch]
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker]
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,[Charles Shyer]


In [ ]:
# Comprueba que se guarden todos los directores en el caso de que una peícula tenga más de uno:
credits[credits["id"]==1725]

,cast,crew,id,director
1838,"[{'cast_id': 6, 'character': 'Maria', 'credit_...","[{'credit_id': '52fe4311c3a36847f8037d3f', 'de...",1725,"[Jerome Robbins, Robert Wise]"


Aplico la funcion *app_ast* a la columna "cast":

In [ ]:
credits["cast"]= credits["cast"].apply(app_ast)

In [ ]:
# Verifico que el tipo de dato:
type(credits["cast"][0])

list

Creo una función que recorra cada diccionario de la lista "cast" y me devuelva una lista con los actores de cada película:

In [ ]:
def get_actors(cast):
    actors=[]
    for member in cast:
        actors.append(member["name"])
    return actors

# Creo una nueva columna en el df "credits" aplicando la función "get_actors" a cada registro de la columna "cast":
credits["actors"]= credits["cast"].apply(get_actors)

# Muestro:

credits.head()

,cast,crew,id,director,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."


Borro los duplicados en *id* , ya que puede darme información incorrecta en una de las consultas de la API:

In [ ]:
credits.id.duplicated().sum() # hay 44 duplicados
credits.drop_duplicates(subset="id", inplace=True)

Me quedo con las columnas de interés para la API y el posterior modelamiento:

In [ ]:
directors = credits.drop(columns=["cast", "crew"], axis=1)
directors.head()

,id,director,actors
0,862,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,8844,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,15602,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,31357,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,11862,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [ ]:
directors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45432 non-null  int64 
 1   director  45432 non-null  object
 2   actors    45432 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


# II. Continuo con los datos de "movies"

Doy una mirada panorámica de *movies*

In [ ]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Reviso los nulos

In [ ]:
movies.info() # De las 45466 fila, tengo solo 4494 no nulos en la columna belongs_to_collection

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Borro los duplicados en id , ya que puede darme información incorrecta en una de las consultas de la API:

In [ ]:
movies.id.duplicated().sum() #hay 30 duplicados
movies.drop_duplicates(subset="id", inplace=True)

De columna "belongs_to" me interesa el "name", ya que éste contiene la franquicia:

In [ ]:
movies["belongs_to_collection"][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

El tipo de dato de cada registro de *belongs_to_collection* es un string de un diccionario, por lo que aplico la función *app_ast*:

In [ ]:
movies["belongs_to_collection"] = movies["belongs_to_collection"].apply(app_ast)

# Confirmo el tipo de dato:
type(movies["belongs_to_collection"][0])


dict

Creo una función para obtener el nombre de la franquicia (serie) a la cual pertenece la película:

In [ ]:
def get_franquicia(serie):
  if pd.notnull(serie) and type(serie)!=float:
    try:
      return serie["name"]
    except (ValueError, SyntaxError):
      return None
  else:
      return None

# Aplico la función a cada registro de la columna y reemplazo la columna:

movies["franquicia"]=movies["belongs_to_collection"].apply(get_franquicia)

# Muestro
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,franquicia
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,None
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection
3,False,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,None
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection


Recorro los registros de "movies" para detectar cuál es el dato tipo float, ya que al querer aplicar la función *app_ast* a la columna *genres*, me devolvió un error diciendo que no podía aplicarse a *floats*:
(*.iterrows()* devuelve pares de "índice- serie(fila)" de un dataframe):

In [ ]:
for index, row in movies.iterrows():
    if isinstance(row["belongs_to_collection"], float):
        print(index, row["belongs_to_collection"])

19730 0.065736
29503 1.931659
35587 2.185485


Inspecciono los registros de estos índices:

In [ ]:
 movies.iloc[[19730, 29503, 35587]]



,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,franquicia
19738,False,None,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,132185,tt0970965,en,Two Days In April,Follows the story of 4 college football player...,...,0.0,90.0,[],Released,NaN,Two Days In April,False,8.0,1.0,None
29523,False,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",NaN,311627,tt4018690,en,Pilot Error,Investigative reporter Nicola Wilson is determ...,...,0.0,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,In the most interconnected world in history it...,Pilot Error,False,0.0,0.0,None
35610,False,None,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,66068,tt0304389,nl,Meisje,Muriel moves from a small village to the big c...,...,0.0,93.0,"[{'iso_639_1': 'nl', 'name': 'Nederlands'}]",Released,NaN,Girl,False,0.0,0.0,None


Estos tres registros tienen los datos en columnas que no les corresponden; al tener popularidad baja y no tener id, decido eliminarlos:

In [ ]:
movies.drop([19730, 29503, 35587], inplace=True )

Ahora sí puedo aplicar la función *app_ast* a la columna *genres* para volverlo en lista, ya que es un sting de una lista:

In [ ]:
# Reviso el tipo de dato:
type(movies["genres"][0])

str

In [ ]:
# Aplico la función:

movies["genres"]=movies["genres"].apply(app_ast)

Creo una función para obtener el nombre de los géneros, ya que el resto de información de esta columna nome es útil:

In [ ]:
def get_genres(genre):
  genres=[]
  for dicc in genre:
    genres.append(dicc["name"])
  return genres

# Modifico la columna genres para que solo tenga una lista con los géneros:
movies["genres"]=movies["genres"].apply(get_genres)

# Muestro
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,franquicia
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,None
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection
3,False,None,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,None
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection


Reviso uno de los registros para confirmar :

In [ ]:
movies["genres"][0]

['Animation', 'Comedy', 'Family']

Continúo con la columna *production_companies*:

In [ ]:
movies["production_companies"].head()

0       [{'name': 'Pixar Animation Studios', 'id': 3}]
1    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2    [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3    [{'name': 'Twentieth Century Fox Film Corporat...
4    [{'name': 'Sandollar Productions', 'id': 5842}...
Name: production_companies, dtype: object

Reviso el tipo de dato:

In [ ]:
movies["production_companies"][0]

"[{'name': 'Pixar Animation Studios', 'id': 3}]"

El tipo de dato es string. Aplico la función *app_ast* para obtener la lista dentro del string:

In [ ]:
movies["production_companies"]=movies["production_companies"].apply(app_ast)

Creo una función para obtener el nombre de las productoras, ya que le resto de la información no me es útil:

In [ ]:
def get_companies(compania):
  companies=[]
  if compania != None and type(compania) != bool:
    try:
      for dicc in compania:
        companies.append(dicc["name"])
      return companies
    except (ValueError, SyntaxError):
      return None
  else:
    return None

# Modifico la columna production_companies para que solo tenga una lista con los productoras:
movies["production_companies"]=movies["production_companies"].apply(get_companies)

# Muestro:
movies["production_companies"].head()

0                            [Pixar Animation Studios]
1    [TriStar Pictures, Teitler Film, Interscope Co...
2                       [Warner Bros., Lancaster Gate]
3             [Twentieth Century Fox Film Corporation]
4         [Sandollar Productions, Touchstone Pictures]
Name: production_companies, dtype: object

Continúo con la columna "production_countries", reviso el tipo de dato:

In [ ]:
type(movies["production_countries"][0])

str

Aplico la función *app_ast* para obtener la lista dentro del sting:

In [ ]:
movies["production_countries"]=movies["production_countries"].apply(app_ast)

Confirmo que se haya aplicado:

In [ ]:
type(movies["production_countries"][0])

list

Creo una función para obtener el nombre de los países, ya que el resto de la información no me es útil:

In [ ]:
movies["production_countries"][0]

[{'iso_3166_1': 'US', 'name': 'United States of America'}]

In [ ]:
def get_countries(countrie):
  countries=[]
  if countrie!= None and type(countrie)!=float:
    try:
      for dicc in countrie:
        countries.append(dicc["name"])
      return countries
    except (ValueError, SyntaxError):
      return None
  else:
      return None


# Modifico la columna para que solo tenga una lista con los países:
movies["production_countries"]=movies["production_countries"].apply(get_countries)

# Muestro:
movies["production_countries"].head()

0    [United States of America]
1    [United States of America]
2    [United States of America]
3    [United States of America]
4    [United States of America]
Name: production_countries, dtype: object

# III. Uno los dataframes

Ya he desanidado todo lo necesario. Ahora, uno los dataframes *credits* y *movies*, y luego me encargo de los nulos:

In [ ]:
# Primero cambio el tipo de dato de la columna movies["id"] y directors["id"]:

movies['id'] = movies['id'].astype('str')
directors['id'] = directors['id'].astype('str')

#Ahora uno a partir del "id":
df_movies = pd.merge(movies, directors, on='id') #how="left")

df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45431
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45432 non-null  object 
 1   belongs_to_collection  4488 non-null   object 
 2   budget                 45432 non-null  object 
 3   genres                 45432 non-null  object 
 4   homepage               7774 non-null   object 
 5   id                     45432 non-null  object 
 6   imdb_id                45415 non-null  object 
 7   original_language      45421 non-null  object 
 8   original_title         45432 non-null  object 
 9   overview               44478 non-null  object 
 10  popularity             45429 non-null  object 
 11  poster_path            45046 non-null  object 
 12  production_companies   45429 non-null  object 
 13  production_countries   45429 non-null  object 
 14  release_date           45345 non-null  object 
 15  re

Cambio el tipo de dato de la columna budget:

In [ ]:
df_movies['budget'] = df_movies['budget'].astype('float')


Imputo nulos por "0" en las columnas *budget* y *revenue*:

In [ ]:
df_movies["budget"] = df_movies["budget"].fillna(0)
df_movies["revenue"] = df_movies["revenue"].fillna(0)

# Reviso que no hayan nulos:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45431
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45432 non-null  object 
 1   belongs_to_collection  4488 non-null   object 
 2   budget                 45432 non-null  float64
 3   genres                 45432 non-null  object 
 4   homepage               7774 non-null   object 
 5   id                     45432 non-null  object 
 6   imdb_id                45415 non-null  object 
 7   original_language      45421 non-null  object 
 8   original_title         45432 non-null  object 
 9   overview               44478 non-null  object 
 10  popularity             45429 non-null  object 
 11  poster_path            45046 non-null  object 
 12  production_companies   45429 non-null  object 
 13  production_countries   45429 non-null  object 
 14  release_date           45345 non-null  object 
 15  re

Elimino nulos de la columna *release_date*:

In [ ]:
df_movies = df_movies.dropna(subset=['release_date'])

# Reviso que no hayan nulos:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45431
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45345 non-null  object 
 1   belongs_to_collection  4485 non-null   object 
 2   budget                 45345 non-null  float64
 3   genres                 45345 non-null  object 
 4   homepage               7761 non-null   object 
 5   id                     45345 non-null  object 
 6   imdb_id                45331 non-null  object 
 7   original_language      45334 non-null  object 
 8   original_title         45345 non-null  object 
 9   overview               44404 non-null  object 
 10  popularity             45345 non-null  object 
 11  poster_path            45006 non-null  object 
 12  production_companies   45345 non-null  object 
 13  production_countries   45345 non-null  object 
 14  release_date           45345 non-null  object 
 15  re

Convierto la columna *release_date* a un formato fecha:

In [ ]:
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')#, errors='coerce'

Confirmo el tipo de dato:

In [ ]:
type(df_movies["release_date"][0])

pandas._libs.tslibs.timestamps.Timestamp

Obtengo una nueva columna con el año:

In [ ]:
df_movies["release_year"] = df_movies['release_date'].dt.year

# Reviso:
df_movies["release_year"].head()

0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: int64

Ahora, creo la columna *return* a partir de *budget* y *revenue*:

In [ ]:
df_movies['return'] = df_movies.apply(lambda row: row['revenue'] / row['budget'] if row['revenue'] and row['budget'] else 0, axis=1)

# Reviso:
df_movies["return"].head()

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
Name: return, dtype: float64

Elimino columnas innecesarias:

In [ ]:
# Creo una lista con las columnas a eliminar:
columnas= ["video" , "imdb_id" , "adult" , "original_title" , "poster_path", "homepage", "belongs_to_collection"]

# Elimino las columnas
df_movies= df_movies.drop(columnas, axis=1)

# Reviso:
df_movies.head()

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,status,tagline,title,vote_average,vote_count,franquicia,director,actors,release_year,return
0,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,...,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",1995,12.451801
1,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,None,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",1995,4.043035
2,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",1995,0.000000
3,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,None,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi...",1995,5.090760
4,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim...",1995,0.000000


Reviso duplicados:

In [ ]:
df_movies["id"].duplicated().sum()

0

No hay duplicados. Ahora reordeno las columnas:

In [ ]:
df_movies.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'franquicia', 'director', 'actors',
       'release_year', 'return'],
      dtype='object')

In [ ]:
# Creo una lista con las columnas en el orden que deseo:
columnas_ordenadas= ['id', 'title', 'original_language',  'genres', 'overview', 'tagline', 'runtime',
                     'spoken_languages', 'status', 'director', 'actors', 'production_companies',
                     'production_countries', 'release_date', 'release_year', 'vote_average', 'vote_count',
                     'popularity', 'franquicia', 'budget', 'revenue', 'return']

# Reordeno las columnas utilizando la lista:
df_movies = df_movies[columnas_ordenadas]

# Reviso:
df_movies.columns

Index(['id', 'title', 'original_language', 'genres', 'overview', 'tagline',
       'runtime', 'spoken_languages', 'status', 'director', 'actors',
       'production_companies', 'production_countries', 'release_date',
       'release_year', 'vote_average', 'vote_count', 'popularity',
       'franquicia', 'budget', 'revenue', 'return'],
      dtype='object')

# IV. Exporto el dataframe final a un csv:

In [ ]:
# Creo la ruta:
savefile_path = '/content/drive/MyDrive/PI_ML_OPS/movies.csv'

# Exporto el dataframe:
df_movies.to_csv(savefile_path, index=False)